# Example use of a schedview accumulated effective time depth map in a Times Square notebook

In [ ]:
# Times Square parameters

night = '2024-11-15'
visit_origin = 'lsstcomcam'

In [ ]:
# Use more recent versions of rubin_scheduler and schedview (will become unnecessary soon)
import sys
sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-11-21/rubin_scheduler')
sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-11-21/schedview')
sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-11-22/rubin_sim')


# Imports
import bokeh.io
import astropy.utils.iers
from schedview import DayObs
from schedview.collect.visits import read_visits, NIGHT_STACKERS
from schedview.plot import create_visit_table
from schedview.plot.survey import create_metric_visit_map_grid
from rubin_sim import maf
from rubin_scheduler.scheduler.model_observatory import ModelObservatory

# Tell astropy we don't need super high precision:
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

# Tell rubin_scheduler where to find its data
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# Prepare jupyter to show bokeh plots
bokeh.io.output_notebook()

In [ ]:
day_obs = DayObs.from_date(night)

In [ ]:
visits = read_visits(day_obs, visit_origin)

In [ ]:
previous_day_obs = DayObs.from_date(day_obs.mjd-1, int_format='mjd')
previous_visits = read_visits(previous_day_obs, visit_origin, stackers=NIGHT_STACKERS, num_nights=10000)

In [ ]:
observatory = ModelObservatory(init_load_length=1)
observatory.mjd = visits.observationStartMJD.median()

In [ ]:
fig = create_metric_visit_map_grid(
        maf.SumMetric(col="eff_time_median", metric_name="Total Teff"),
        previous_visits.loc[previous_visits.eff_time_median > 0, :],
        visits,
        observatory,
        nside=32,
        use_matplotlib=True
    )

Each subplot represents the progress in the survey in a different filter, presented in a [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole.

 - Black outlines show the camera footprint of each exposure taken on this night.
 - The blue backdrop represents the accumulated effective exposure time, t<sub>eff</sub> (as defined above) at the start of the night.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
 - The blue line (oval) shows the plane of the Milky Way.
 - Broken red ovals show a zenith distince of 70° (airmass=2.9) at morning and evening 12° twilight.
   - The dashed portions of each red oval show the extent of what is observable at all during the night.
   - As time progress through the night, the ZD circle maintains its shape, moving counter-clockwise around the pole over the course of the night.

Parts of the sky visited most recently are dark blue, growing fainter and grayer for areas visited less recently. Regions not visited in the last 10 days are not colored.
